# FuLeeca Attack modeling using Pyomo

In this notebook, we are modelling the FuLeeca attack in Pyomo so that we will be able to use different solvers instead of just Gurobi.

Solver that did give a forgesk for toyexample.

- Highs 8.5 s
- Gurobi (needs actual licence)
- Couenne 4838.0 s (maybe we should not use this)


Solvers that did give error

- ipopt: max iter problem
- cbc could not run the toy example after 1 hour
- amplgsl does not woork
- baron does not work 
- base does not work
- conopt did not give correct output
- copt did not work
-  'ilogcp' it ios for scheduling so do not use it
- cplex did not give an output
- gcg took too long
- gecode: not for continuous variable


In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# #Loading the libraries to be used 
# from pyomo.environ import *
# import matplotlib.pyplot as plt
# import random 
# import numpy as np
# import pandas as pd
# from pyomo.common.timing import TicTocTimer
# from amplpy import modules
# from pyomo.opt import SolverFactory

# model = AbstractModel()
# model.level = Param()
# model.halfn = Param()
# model.p = Param()
# model.uba = Param() #upper bound of a     
# model.i = Set()
# model.j = RangeSet(1, model.halfn)
# model.T = Param(model.i,model.j, within=Reals)
# model.Mset = Param(model.i, within=Reals)

# def initval(model,i):
#     return random.randint(-model.uba,model.uba) #initialization of the variable
# def init0(model,i):
#     return(0)
# def init1(model,i,j):
#     return(0)
# model.a = Var(model.i, bounds=(-model.uba,model.uba), within= Reals, initialize=initval)
# model.b = Var(model.i, bounds=(-model.uba,model.uba) , within= Reals, initialize=initval)
# model.q = Var(model.i, within=Integers,initialize=init0)
# model.Pa = Var(model.i,model.j,  bounds=(-1,1), within=Integers,initialize=init1) 
# model.Pb = Var(model.i,model.j,  bounds=(-1,1), within=Integers,initialize=init1) 

# def rule_baT(model,i):
#         return(np.sum([model.a[j]*model.T[j,i] for j in model.j ]) == model.q[i]*model.p + model.b[i])#b =aT    
# model.C1   = Constraint(model.i,rule=rule_baT)

# def rule_a(model,i):
#     return(sum([model.Mset[j]*model.Pa[j,i] for j in model.j]) == model.a[i])#building a
# model.Ca = Constraint(model.i, rule = rule_a)
# def rule_b(model,i):
#     return(sum([model.Mset[j]*model.Pb[j,i] for j in model.j]) == model.b[i])#building a
# model.Cb = Constraint(model.i, rule = rule_b)

# def rule_cPa(model,i):
#     return((sum([(model.Pa[j,i])**2  for j in model.j]))==1)#each column exactly one nonzero elements  
# model.cPa = Constraint(model.i, rule = rule_cPa)
# def rule_cPb(model,i):
#     return((sum([(model.Pb[j,i])**2 for j in model.j])) ==1)#each column has exactly one nonzero elements  
# model.cPb = Constraint(model.i, rule = rule_cPb)
# def rule_rPa(model,i):
#     return((sum([(model.Pa[i,j])**2 for j in model.j])) ==1)#each row exactly one nonzero elements  
# model.rPa = Constraint(model.i, rule = rule_rPa)
# def rule_rPb(model,i):
#     return((sum([(model.Pb[i,j])**2 for j in model.j])) ==1)#each row has exactly one nonzero elements  
# model.rPb = Constraint(model.i, rule = rule_rPb)

# def rule_OF(model):
#     return sum(model.q[i] for i in model.i)

# model.obj = Objective(rule=rule_OF, sense=maximize)
# #opt.options['max_iter'] = 100000

# #['bonmin', 'cbc', 'conopt', 'couenne', 'cplex', 'filmint', 'filter', 'ipopt', 'knitro', 'l-bfgs-b', 
# #'lancelot', 'lgo', 'loqo', 'minlp', 'minos', 'minto', 'mosek', 'octeract', 'ooqp', 'path', 'raposa', 'snopt']

In [ ]:
print(modules.installed())

In [ ]:
print(modules.find('highs'))

In [ ]:
modules.install('gurobi')

In [ ]:
# def forge_sk(filename = "toyexample.dat", store_at = "forgedsk.txt", solvername = 'highs',verbose = False):
#     opt = SolverFactory(modules.find(solvername), solve_io="nl") #Couenne was good, ipopt did not give a result(always max iteration)
#     timer = TicTocTimer()
#     timer.tic('starting timer')
#     instance = model.create_instance(filename)
#     print('We just have built the instance, start solving stay tuned!')
#     results = opt.solve(instance,keepfiles=True, logfile="my.log")

    
#     if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
#         print('feasible')
#     elif (results.solver.termination_condition == TerminationCondition.infeasible):
#         print('infeasible')
#     else:
#         print ('Solver Status:',  results.solver.status)
#     print(value(instance.obj))
#     a = [int(round(instance.a[i].value,0)) for i in instance.i]
#     b = [int(round(instance.b[i].value,0)) for i in instance.i]
#     sk = a+b
#     T = [[instance.T[i,j] for j in instance.j] for i in instance.i]
#     p = instance.p.value
#     print('Verifying that indeed aT = b mod p \n This should be 1 if the sekret keys are valid:\n')
#     isokay = prod(np.dot(a,T) %p == np.array(b)%p)
#     print(isokay)
#     #Write a and b in a file 
#     np.savetxt(store_at, sk, fmt="%d", delimiter=",")
#     dT = timer.toc('task 1')
#     print("elapsed time: %0.1f s" % dT)
#     if verbose:
#         print('Value of a' + str(a))
#         print('Value of b' + str(b))
#     return([isokay,dT])

In [ ]:
# forge_sk(solvername='gurobi')

In [ ]:
#level 1 
#forge_sk(filename = "level1example.dat", store_at = "forgedlevel1sk.txt", verbose = False)

In [ ]:
#Toyexample with Highs
#forge_sk(filename = "toyexample.dat", store_at = "toyhighssk.txt", verbose = False)

In [ ]:
listsolver = ['highs']

In [ ]:
# usedtime = []
# for solver in listsolver:
#     print("forging the toyexample using " +solver)
#     try:
#         usedtime.append([solver]+
#                         forge_sk(filename = "toyexample.dat",
#                                  store_at = "toyhighs"+solver+"sk.txt",
#                                  solvername = solver, 
#                                  verbose = False))
#     except:
#         continue

In [ ]:
usedtime

## Attack idea #2

This is another mathematically equivalent formulation of the previous model. We are going to see how fast it is comparatively to the previous model.

This is a linear model. Basically instead of using a = Pa Mset, we split a into a+ and a-. We will have mroe variables, and more constraints but will have all linear equations. We hope being able to solve this using Gurobi.



In [2]:
#Loading the libraries to be used 
from pyomo.environ import *
import matplotlib.pyplot as plt
import random 
import numpy as np
import pandas as pd
from pyomo.common.timing import TicTocTimer
from amplpy import modules

model = AbstractModel()
model.level = Param()
model.halfn = Param()
model.p = Param()
model.uba = Param() #upper bound of a     
model.i = Set()
model.j = RangeSet(1, model.halfn)
model.T = Param(model.i,model.j, within=Reals)
model.Mset = Param(model.i, within=Reals)

def initval(model,i):
    return random.randint(-model.uba,model.uba) #initialization of the variable
def init0(model,i):
    return(0)
def init1(model,i,j):
    return(0)
model.a = Var(model.i, bounds=(-model.uba,model.uba), within= Reals, initialize=init0)
model.ap = Var(model.i, bounds=(0,model.uba), within= Reals, initialize=init0)
model.am = Var(model.i, bounds=(-model.uba,0), within= Reals, initialize=init0) # am
model.b = Var(model.i, bounds=(-model.uba,model.uba) , within= Reals, initialize=init0)
model.bp = Var(model.i, bounds=(0,model.uba) , within= Reals, initialize=init0)
model.bm = Var(model.i, bounds=(-model.uba,0) , within= Reals, initialize=init0)
model.q = Var(model.i, within=Integers,initialize=init0)
model.Pap = Var(model.i,model.j,  bounds=(0,1), within=Integers,initialize=init1) 
model.Pbp = Var(model.i,model.j,  bounds=(0,1), within=Integers,initialize=init1) 
model.Pam = Var(model.i,model.j,  bounds=(0,1), within=Integers,initialize=init1) 
model.Pbm = Var(model.i,model.j,  bounds=(0,1), within=Integers,initialize=init1) 

def rule_baT(model,i): #b =aT
        return(np.sum([model.a[j]*model.T[j,i] for j in model.j ]) == model.q[i]*model.p + model.b[i])#b =aT    
model.C1   = Constraint(model.i,rule=rule_baT)

def rule_am(model,i): #am = -Mset*Pam
    return(sum([-model.Mset[j]*model.Pam[j,i] for j in model.j]) == model.am[i])#building a
model.Cam = Constraint(model.i, rule = rule_am)
def rule_ap(model,i): #ap = Mset*Pap
    return(sum([model.Mset[j]*model.Pap[j,i] for j in model.j]) == model.ap[i])#building a
model.Cap = Constraint(model.i, rule = rule_ap)

def rule_sum_a(model,i): #ap = Mset*Pap
    return(model.am[i]+ model.ap[i]== model.a[i])#building a
model.sum_a = Constraint(model.i, rule = rule_sum_a)

def rule_bm(model,i): #am = -Mset*Pam
    return(sum([-model.Mset[j]*model.Pbm[j,i] for j in model.j]) == model.bm[i])#building a
model.Cbm = Constraint(model.i, rule = rule_bm)
def rule_bp(model,i): #ap = Mset*Pap
    return(sum([model.Mset[j]*model.Pbp[j,i] for j in model.j]) == model.bp[i])#building a
model.Cbp = Constraint(model.i, rule = rule_bp)

def rule_sum_b(model,i): #ap = Mset*Pap
    return(model.bm[i]+ model.bp[i]== model.b[i])#building a
model.sum_b = Constraint(model.i, rule = rule_sum_b)



def rule_cPa(model,i):
    return((sum([(model.Pam[j,i])  for j in model.j])) + (sum([(model.Pap[j,i])  for j in model.j]))==1)#each column exactly one nonzero elements  
model.cPa = Constraint(model.i, rule = rule_cPa)

def rule_cPb(model,i):
    return((sum([(model.Pbm[j,i]) for j in model.j])) + (sum([(model.Pbp[j,i]) for j in model.j]))==1)#each column has exactly one nonzero elements  
model.cPb = Constraint(model.i, rule = rule_cPb)

def rule_rPa(model,i):
    return((sum([(model.Pam[i,j]) for j in model.j])) +(sum([(model.Pap[i,j]) for j in model.j])) ==1)#each row exactly one nonzero elements  
model.rPa = Constraint(model.i, rule = rule_rPa)
def rule_rPb(model,i):
    return((sum([(model.Pbm[i,j]) for j in model.j])) + (sum([(model.Pbp[i,j]) for j in model.j]))==1)#each row has exactly one nonzero elements  
model.rPb = Constraint(model.i, rule = rule_rPb)


def rule_OF(model):
    return sum(model.q[i] for i in model.i)

model.obj = Objective(rule=rule_OF, sense=maximize)
#opt.options['max_iter'] = 100000

#['bonmin', 'cbc', 'conopt', 'couenne', 'cplex', 'filmint', 'filter', 'ipopt', 'knitro', 'l-bfgs-b', 
#'lancelot', 'lgo', 'loqo', 'minlp', 'minos', 'minto', 'mosek', 'octeract', 'ooqp', 'path', 'raposa', 'snopt']

In [3]:
def forge_lin_sk(filename = "toyexample.dat", store_at = "forgedsk.txt", solvername = 'highs', ampl = True, verbose = False):
    if ampl==True:
        opt = SolverFactory(modules.find(solvername), solve_io="nl") #Couenne was good, ipopt did not give a result(always max iteration)
    else:
        opt = SolverFactory(solvername)
    if solvername == 'gurobi':
        # put some option here to focus on feasiblility
        opt.options['MIPFocus'] =1 #focus on feasibility
        opt.options['ZeroObjNodes'] =5  #we want to run an heuristic search if the model did not find anything
        opt.options['IntegralityFocus'] =1 #Please take the integrality seriously
        opt.options['SubMIPCuts'] =2
    timer = TicTocTimer()
    timer.tic('starting timer')
    instance = model.create_instance(filename)
    print('We just have built the instance, start solving stay tuned!')
    results = opt.solve(instance,keepfiles=True, logfile="my.log")

    
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        print('feasible')
    elif (results.solver.termination_condition == TerminationCondition.infeasible):
        print('infeasible')
    else:
        print ('Solver Status:',  results.solver.status)
    print(value(instance.obj))
    a = [int(round(instance.a[i].value,0))%p for i in instance.i]
    b = [int(round(instance.b[i].value,0))%p for i in instance.i]
    sk = a+b
    T = [[instance.T[i,j] for j in instance.j] for i in instance.i]
    p = instance.p.value
    print('Verifying that indeed aT = b mod p \n This should be 1 if the sekret keys are valid:\n')
    prodat = np.dot(a,T) %p
    isokay = prod(prodat == np.array(b))
    print(isokay)
    #Write a and b in a file 
    np.savetxt(store_at, sk, fmt="%d", delimiter=",")
    dT = timer.toc('task 1')
    print("elapsed time: %0.1f s" % dT)
    if verbose:
        print('Value of a' + str(a))
        print('Value of b' + str(b))
        print('Value of aT' + str(prodat))
    return([isokay,dT])

In [4]:
forge_lin_sk(solvername ='gurobi', ampl = False,verbose = True)#Toy examnple

[    0.00] starting timer
We just have built the instance, start solving stay tuned!
Solver script file: 'C:\Users\18645\AppData\Local\Temp\tmp84ctnfrm.gurobi.script'
Solver log file: 'my.log'
Solver solution file: 'C:\Users\18645\AppData\Local\Temp\tmpockrzwbb.gurobi.txt'
Solver problem files: ('C:\\Users\\18645\\AppData\\Local\\Temp\\tmpgy3trcdk.pyomo.lp',)
feasible
59.0
Verifying that indeed aT = b mod p 
 This should be 1 if the sekret keys are valid:

1
[+   0.91] task 1
elapsed time: 0.9 s
Value of a[0, 2, 1, 0, 1, 1, 0, 2, 2, 1, 1]
Value of b[0, 0, 1, -2, -1, -1, -1, -2, -1, 0, -2]
Value of aT[0 0 1 3 4 4 4 3 4 0 3]


[1, 0.9063051000000044]

In [6]:
#forge_lin_sk(solvername ='highs', ampl = 1,verbose = True) #does not work on my machine

In [ ]:
forge_lin_sk(solvername ='gurobi', ampl = False, verbose = True, filename = 'level42example.dat', store_at = 'forgesk42.txt')

[    0.00] starting timer
We just have built the instance, start solving stay tuned!
Solver script file: 'C:\Users\18645\AppData\Local\Temp\tmpv8adjkvc.gurobi.script'
Solver log file: 'my.log'
Solver solution file: 'C:\Users\18645\AppData\Local\Temp\tmp459jkjg3.gurobi.txt'
Solver problem files: ('C:\\Users\\18645\\AppData\\Local\\Temp\\tmpuhm4uuiw.pyomo.lp',)
